In [1]:
from tqdm.auto import tqdm
import os

import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split , StratifiedKFold


import tensorflow as tf 
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Input,Dense, LSTM, RNN, Bidirectional, GlobalAveragePooling2D , Dropout, Conv1D, Flatten
from tensorflow.keras.utils import to_categorical

from transformers import TFAutoModel , AutoTokenizer

from sklearn.metrics import classification_report

In [2]:
col_names = ['labels','text']
df_tamil_mine = pd.read_csv('../input/cross-verifying-results/BpHigh_tamil.tsv',sep ='\t')
df_tamil_en_mine = pd.read_csv('../input/cross-verifying-results/BpHigh_tamil-english.tsv',sep ='\t')
df_tamil_given = pd.read_csv('../input/cross-verifying-results/ta-misogyny-test.csv',names=col_names,sep ='\t')
df_tamil_en_given = pd.read_csv('../input/cross-verifying-results/ta-en-misogyny-test.csv',names=col_names,sep ='\t')

In [3]:
def transform_df_labels(df):
    df = df.replace({'Counter-speech':0,
                             'Homophobia':1, 
                             'Hope-Speech':2, 
                             'Misandry':3, 
                             'Misogyny':4, 
                             'None-of-the-above':5, 
                             'Transphobic':6,
                             'Xenophobia':7})
    return df

In [4]:
df_tamil_mine

,Unnamed: 0,text,label
0,0,அந்த தமிழன் ஒரு சாதிவெறி,None-of-the-above
1,1,இந்த வந்தேரிக்கு தகுந்த பாடம் கற்பிப்பார்கள் த...,None-of-the-above
2,2,எச்ச ராஜா நீ ஒரு மானங்கெட்ட தாயை விட நீ,Misandry
3,3,பணம் இருக்கும் இடத்தில் கேட்பது தவறு இல்லை பண...,None-of-the-above
4,4,சக தோழியாக நினைத்து ஆதரவு தாருங்கள் நண்பர்களே!...,None-of-the-above
...,...,...,...
694,694,சினிவாசன் அய்யாக்கு நன்றி...,None-of-the-above
695,695,ஊருக்கே தெரியும் அவர் பிராமணன் என்று.அவர் ஒரு தலை,None-of-the-above
696,696,ஐயா அவர்கள் சரியான விளக்கம் பார்பனர்கள் வேறு ப...,None-of-the-above
697,697,Nandha Kumar அட நாயே; ; வந்தேறி பிராமண நாய...,Counter-speech


In [5]:
df_tamil_given.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   labels  700 non-null    object
 1   text    699 non-null    object
dtypes: object(2)
memory usage: 11.1+ KB


In [6]:
null_list = df_tamil_given[df_tamil_given['text'].isnull()].index.tolist()

In [7]:
df_tamil_given.dropna(how='any',axis=0,inplace=True)

In [8]:
df_tamil_given

,labels,text
0,Xenophobia,அந்த தமிழன் ஒரு சாதிவெறி
1,Xenophobia,இந்த வந்தேரிக்கு தகுந்த பாடம் கற்பிப்பார்கள் த...
2,Misandry,எச்ச ராஜா நீ ஒரு மானங்கெட்ட தாயை விட நீ
3,Counter-speech,பணம் இருக்கும் இடத்தில் கேட்பது தவறு இல்லை பண...
4,None-of-the-above,சக தோழியாக நினைத்து ஆதரவு தாருங்கள் நண்பர்களே!...
...,...,...
695,None-of-the-above,சினிவாசன் அய்யாக்கு நன்றி...
696,None-of-the-above,ஊருக்கே தெரியும் அவர் பிராமணன் என்று.அவர் ஒரு தலை
697,None-of-the-above,ஐயா அவர்கள் சரியான விளக்கம் பார்பனர்கள் வேறு ப...
698,Misogyny,Nandha Kumar அட நாயே; ; வந்தேறி பிராமண நாய...


In [9]:
df_tamil_en_given.dropna(how='any',axis=0,inplace=True)

In [10]:
df_tamil_mine_transformed = transform_df_labels(df_tamil_mine['label'])
df_tamil_given_transformed = transform_df_labels(df_tamil_given['labels'])
df_tamil_en_mine_transformed = transform_df_labels(df_tamil_en_mine['label'])
df_tamil_en_given_transformed = transform_df_labels(df_tamil_en_given['labels'])


In [11]:
target_names = ['Counter-speech',
                             'Homophobia', 
                             'Hope-Speech', 
                             'Misandry', 
                             'Misogyny', 
                             'None-of-the-above', 
                             'Transphobic',
                             'Xenophobia']
print(classification_report(df_tamil_given_transformed ,df_tamil_mine_transformed,target_names=target_names))

                   precision    recall  f1-score   support

   Counter-speech       0.13      0.04      0.06        47
       Homophobia       0.00      0.00      0.00         8
      Hope-Speech       0.00      0.00      0.00        26
         Misandry       0.52      0.45      0.48       127
         Misogyny       0.00      0.00      0.00        48
None-of-the-above       0.68      0.94      0.79       416
      Transphobic       0.00      0.00      0.00         2
       Xenophobia       0.00      0.00      0.00        25

         accuracy                           0.64       699
        macro avg       0.17      0.18      0.17       699
     weighted avg       0.51      0.64      0.56       699



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
print(classification_report(df_tamil_en_given_transformed ,df_tamil_en_mine_transformed,target_names=target_names))

                   precision    recall  f1-score   support

   Counter-speech       0.29      0.18      0.22        88
       Homophobia       0.00      0.00      0.00        56
      Hope-Speech       0.00      0.00      0.00        70
         Misandry       0.54      0.49      0.52       292
         Misogyny       0.00      0.00      0.00        57
None-of-the-above       0.71      0.95      0.81      1141
      Transphobic       0.00      0.00      0.00        58
       Xenophobia       0.00      0.00      0.00        95

         accuracy                           0.67      1857
        macro avg       0.19      0.20      0.19      1857
     weighted avg       0.53      0.67      0.59      1857

